Load modules

In [1]:
from pytube import YouTube as yt
import os
import cv2
from tqdm import tqdm
import numpy as np
from PIL import Image

URL

In [2]:
train_url = "https://www.youtube.com/watch?v=5EXgtz_seFA"    # 극장판 1기 예고편 (무한열차편)
test_url = "https://www.youtube.com/watch?v=MQ9KbWtur5U"     # 극장판 2기 예고편 (유곽편)

Download

In [3]:
def download(url : str):
  streams = yt(url).streams
  best_stream = streams.get_highest_resolution()
  if best_stream == None:
    raise Exception("Down 실패")
  else:
    best_stream.download()
    print("Down 성공")

train_stream = download(train_url)
test_stream = download(test_url)

Down 성공
Down 성공


Get gray clip

In [4]:
train_clip_path = '[한글자막] 귀멸의칼날 무한열차편 4차 PV 예고편.mp4'
test_clip_path = '【풀버전】 귀멸의 칼날 2기(유곽편) 예고편(PVTrailer) 한국어 번역 KORENGJPN.mp4'

In [7]:
def preprocess(cilp_path : str):

    capture = cv2.VideoCapture(cilp_path) 
    
    n_frame = int(capture.get(cv2.CAP_PROP_FRAME_COUNT))
    height = 360  
    width = 640    

    color_stack = []
    gray_stack = []

    for idx in tqdm(range(n_frame)):
      
      ret, original_frame = capture.read()
      
      if not ret:
        break

      resized_frame = cv2.resize(original_frame, (width, height), interpolation = cv2.INTER_AREA)
      rgb_frame = cv2.cvtColor(resized_frame, cv2.COLOR_BGR2RGB)
      gray_frame = cv2.cvtColor(rgb_frame, cv2.COLOR_BGR2GRAY)

      color_stack.append(rgb_frame)
      gray_stack.append(gray_frame)

    return  np.stack(gray_stack),np.stack(color_stack)

In [8]:
train_x, train_y = preprocess(train_clip_path)

100%|██████████| 2562/2562 [00:09<00:00, 277.33it/s]


In [ ]:
test_x, test_y = preprocess(test_clip_path)

numpy.ndarray

In [9]:
np.savez_compressed('train.npz', train_x = train_x, train_y = train_y)

In [ ]:
np.savez_compressed('test.npz', test_x = test_x, test_y = test_y)